<a href="https://colab.research.google.com/github/xingleiking/iodraw-files/blob/main/Prime_TM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 安装依赖 (在Colab中新单元格)
!pip install transformers biopython torch pandas tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 57.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [2]:
# TM prediction with Prime

# 导入库
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd
from Bio import SeqIO
from tqdm.notebook import tqdm
from google.colab import files

In [3]:
# 上传FASTA文件
print("请上传FASTA文件:")
uploaded = files.upload()
sequence_file = list(uploaded.keys())[0]  # 获取上传的文件名

# 或者创建一个示例FASTA文件
if not uploaded:
    with open("example.fasta", "w") as f:
        f.write(">sample_protein1\nMKTIIALSYIFCLVFAKHK\n>sample_protein2\nMKTIIALSYIFCLVFA")
    sequence_file = "example.fasta"
    print("使用示例FASTA文件")





请上传FASTA文件:


Saving example.fasta to example.fasta


In [4]:
# 加载模型和分词器
model_path = "AI4Protein/Prime_690M"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用设备: {device}")

try:
    model = AutoModel.from_pretrained(model_path, trust_remote_code=True)
    tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
    model.eval()
    model = model.to(device)
    print("模型加载成功!")
except Exception as e:
    print(f"加载模型时出错: {e}")

使用设备: cpu


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/894 [00:00<?, ?B/s]

configuration_proprime.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/AI4Protein/Prime_690M:
- configuration_proprime.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

tokenization_proprime.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/AI4Protein/Prime_690M:
- tokenization_proprime.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


vocab.txt:   0%|          | 0.00/93.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

模型加载成功!


In [5]:
# 预测
tm = []
records = list(SeqIO.parse(sequence_file, "fasta"))
if not records:
    print("FASTA文件中没有找到序列!")
else:
    with torch.no_grad():
        for record in tqdm(records, desc="预测中"):
            try:
                sequence = str(record.seq)
                tokenied_results = tokenizer(sequence, return_tensors="pt")
                input_ids = tokenied_results.input_ids.to(device)
                attention_mask = tokenied_results.attention_mask.to(device)
                logits = model(input_ids=input_ids, attention_mask=attention_mask).predicted_values
                tm.append(logits.item())
            except Exception as e:
                print(f"处理序列 {record.id} 时出错: {e}")
                tm.append(None)

    # 显示结果
    results = pd.DataFrame({
        "Protein ID": [record.id for record in records],
        "Predicted TM": tm
    })
    print("\n预测结果:")
    print(results)

预测中:   0%|          | 0/14 [00:00<?, ?it/s]


预测结果:
    Protein ID  Predicted TM
0       Q544R8     29.424969
1       Q91YN8     30.343319
2       Q9ZSA2     25.358501
3       F8WHX2     28.985481
4       Q8ML92     25.786642
5       P56801     25.800392
6   A0A0R4J0T5     25.850422
7       Q8BYW9     28.526733
8       P37470     30.309776
9       Q9W328     22.676493
10      Q6ZNW5     23.928360
11      A6NDG6     25.096235
12      C9JSR1     21.365665
13      Q6P1B9     27.363907
